# Build train and test matrices

In [175]:
import pandas as pd
import numpy as np
import feather
import pandas_profiling
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

df = (feather.read_dataframe('/home/SHARED/SOLAR/data/oahu_min.feather')
             .set_index('Datetime'))

In [176]:
# We load the info of the sensors to extract the longitude information
info = pd.read_csv('/home/SHARED/SOLAR/data/info.csv')

info.Location = info.Location.apply(lambda x: (x[:2] + x[-2:]).replace('_', ''))
info.index = info.Location
# Sorted longitudes
longs = info['Longitude'].sort_values(ascending=False)

# We drop two sensors (they are different compared to the other 17, since they are "tilted")
df.drop('GT_AP6', inplace=True, axis=1)
df.drop('GT_DH1', inplace=True, axis=1)

# Just some auxiliar code to homogeneize name of sensors across different tables
homogen_name = lambda x: x[-4:].replace('_', '')
df.columns = [homogen_name(x) for x in df.columns.values.tolist()]

# Finally, we sort the data according to sensor's longitude
df = df[longs.index]

In [177]:
# https://stackoverflow.com/questions/15722324/sliding-window-in-numpy
def window_stack_forward(a, stepsize=1, width=3):
    return np.hstack( a[i:1+i-width or None:stepsize] for i in range(0, width) )

In [178]:
# I feel this function can also be done for pd.DataFrame
def window_stack(a, width=3):
    n = a.shape[0]
    return np.hstack(list(a[(width-1-i):(n-i)] for i in range(0, width)))

In [179]:
# In pandas 0.24, use df.to_numpy() instead of df.values. Also care with non-numeric columns
width = 61
a = window_stack(df.to_numpy(), width=width)

In [180]:
times   = [ ('t' if not idx else 't-{:d}'.format(idx)) for idx in range(width) ]
columns = pd.MultiIndex.from_product((times, df.columns), names=('time', 'location'))

In [181]:
# Convert back to DataFrame, just for convenience of having indexes
df_roll = pd.DataFrame(a, index=df.index[width-1:], columns=columns)

In [182]:
# pandas_profiling.ProfileReport(df_roll)

In [183]:
# Split target (time t) and variables (times t-1 to t-width+1)
y = df_roll['t']
X = df_roll.drop(columns='t', level='time')

In [184]:
# Split train-test, approximately 12 and 4 months respectively
X_train, X_test = X[:'2011-07-31'], X['2011-08-01':]
y_train, y_test = y[:'2011-07-31'], y['2011-08-01':]

In [185]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(449885, 1020)
(82892, 1020)
(449885, 17)
(82892, 17)


# Convolutional predictor

First we preprocess the dataset (for the moment, we'll just use as features the t-1 values at each sensor)

In [15]:
# We only use the previous timestep as features
X_tr1 = X_train['t-1']
y_tr1 = y_train

X_te1 = X_test['t-1']
y_te1 = y_test

Now, in order to use a 1D convolution, we are going to sort the sensors. For the initial test, we'll just sort them by longitude (from East to West). That way, nearer sensors are in close positions in the tensor, so the 1D convolution may extract useful correlations.

Note: many other possible ordenations of the sensors could be added as new channels in the input tensor

In [16]:
# We load the info of the sensors to extract the longitude information
info = pd.read_csv('/home/SHARED/SOLAR/data/info.csv')

info.Location = info.Location.apply(lambda x: (x[:2] + x[-2:]).replace('_', ''))
info.index = info.Location
# Sorted longitudes
longs = info['Longitude'].sort_values(ascending=False)

# We drop two sensors (they are different compared to the other 17, since they are "tilted")
X_tr1.drop('GT_AP6', inplace=True, axis=1)
y_tr1.drop('GT_AP6', inplace=True, axis=1)
X_tr1.drop('GT_DH1', inplace=True, axis=1)
y_tr1.drop('GT_DH1', inplace=True, axis=1)
X_te1.drop('GT_AP6', inplace=True, axis=1)
y_te1.drop('GT_AP6', inplace=True, axis=1)
X_te1.drop('GT_DH1', inplace=True, axis=1)
y_te1.drop('GT_DH1', inplace=True, axis=1)

# Just some auxiliar code to homogeneize name of sensors across different tables
homogen_name = lambda x: x[-4:].replace('_', '')
X_tr1.columns = [homogen_name(x) for x in X_tr1.columns.values.tolist()]
y_tr1.columns = [homogen_name(x) for x in y_tr1.columns.values.tolist()]
X_te1.columns = [homogen_name(x) for x in X_te1.columns.values.tolist()]
y_te1.columns = [homogen_name(x) for x in y_te1.columns.values.tolist()]


# Finally, we sort the data according to sensor's longitude
X_tr1_1 = X_tr1[longs.index]
y_tr1_1 = y_tr1[longs.index]
X_te1_1 = X_te1[longs.index]
y_te1_1 = y_te1[longs.index]

/home/davidgg/anaconda3/envs/solar/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Now we specify which sensor do we want to predict and test.

(In the future, we need to discuss how are we going to predict, if just by looping over each sensor, or just give a vectorial prediction)

In [28]:
import keras

from keras.datasets import cifar10
# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape, Add, Multiply, Subtract, Dropout
from keras.layers import Conv2D, MaxPooling2D, LocallyConnected1D, Conv1D, UpSampling1D, MaxPooling1D, Dot, Concatenate
from keras.layers import LocallyConnected2D, Conv2D
from keras import backend as K

In [2]:

# from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential, Model
# from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape, Add, Multiply, Subtract, Dropout
# from keras.layers import Conv2D, MaxPooling2D, LocallyConnected1D, Conv1D, UpSampling1D, MaxPooling1D, Dot, Concatenate

# from keras import backend as K

Model architecture is defined below.

Some highlights:
* Locally connected works better than pure convolutional at the first layers (probably because the sensors at not located in a uniform grid)
* Trick to improve acc: add a final layer combining the convolutional prediction with the persistance prediction, so in case the input is "strange", the model could learn to output the persistance prediction (i.e., the previous time-step), which is somewhat reasonable

In [17]:
def make_model_sensor(index_sensor, n_sensors=17):
    ''' Returns a model using all the sensors to predict index_sensor '''
    xin = Input(shape=(n_sensors,1), name='main_input')
    x = LocallyConnected1D(8, 7, data_format = 'channels_last', padding='valid')(xin)
    x = Activation('relu')(x)
    x = LocallyConnected1D(16, 5, data_format = 'channels_last', padding='valid')(x)
    x = Activation('relu')(x)
    x = Conv1D(32, 3, data_format = 'channels_last', padding='causal')(x)
    xl = Flatten()(x)
    xl = Dropout(0.2)(xl)
    xo = Dense(1)(xl)

    # use date info here?
    xinf = Flatten()(xin)
    s  = Dense(5)(xinf)
    s = Activation('tanh')(s)
    s = Dense(2)(s)
    s = Activation('softmax')(s)

    # sort of residual connection
    xin_0 = Activation('relu')(xin)
    xin_1 = Lambda(lambda x : x[:,index_sensor,:])(xin_0)
    xo_m = Dot(axes=1)([Concatenate()([xo,xin_1]), s])
    xo_m = Activation('relu')(xo_m)

    model = Model(inputs=[xin], outputs=[xo_m])
    return model

In [18]:
lr = 0.0001
lr = 0.0001
opt = keras.optimizers.Adam(lr=lr)

# We add a callback to log metrics and another one to schedule the learning rate

#see clr.py in this same folder
from clr import CyclicLR

c1 = keras.callbacks.BaseLogger(stateful_metrics=None)
c2 = CyclicLR(step_size=250, base_lr=lr)
c3 = keras.callbacks.History()

Now we are ready to train. The below configuration should take 2 minutes in a 16 core CPU
(no GPU needed). We are using a huge batch-size to speed up things

In [19]:
n_sensors = 17

def to_array(sensor='AP5', val=0.1):
    ''' Converts dataframe to numpy array for predicting any given sensor. val specifies the fraction
    of training samples to be used as validation. '''
    X_tr1_1_np = X_tr1_1.values
    y_tr1_1_np = y_tr1_1[sensor].values
    
    #val_idx = int((1 - val)*len(y_tr1_1_np))

    X_te1_1_np = X_te1_1.values
    y_te1_1_np = y_te1_1[sensor].values
    
    #return X_tr1_1_np[:val_idx], y_tr1_1_np[:val_idx], X_tr1_1_np[val_idx:], y_tr1_1_np[val_idx:], X_te1_1_np, y_te1_1_np
    return X_tr1_1_np, y_tr1_1_np, X_te1_1_np, y_te1_1_np

In [20]:
batch_size = 1 << 11   # as big as possible so we can explore many models
epochs = 1 << 5

In [22]:
from sklearn.model_selection import TimeSeriesSplit

longs_np = longs.index.values

In [23]:
def train_and_test_sensor(id_sensor=4):
    X_tr, y_tr, X_te, y_te = to_array(sensor=longs_np[id_sensor])
    
    
    # Validation using TS split (just to obtain different MAE estimations, no hyperoptimization for the moment)
    for tr_idx, va_idx in TimeSeriesSplit(n_splits=5).split(X_tr):
        model = make_model_sensor(id_sensor, n_sensors=17)
        model.compile(opt, loss='mean_absolute_error')
        model.fit(np.atleast_3d(X_tr[tr_idx]), y_tr[tr_idx], batch_size=batch_size, epochs=epochs, validation_data=
              (np.atleast_3d(X_tr[va_idx]),y_tr[va_idx]), callbacks=[c2, c3], verbose=0)
        print('MAE_val ', c3.history['val_loss'][-1])
    
    # Testing
    model = make_model_sensor(id_sensor, n_sensors=17)
    model.compile(opt, loss='mean_absolute_error')
    model.fit(np.atleast_3d(X_tr), y_tr, batch_size=batch_size, epochs=epochs, validation_data=
              (np.atleast_3d(X_te),y_te), callbacks=[c2, c3], verbose=0)
    
    print('MAE_test ', c3.history['val_loss'][-1])
    return longs_np[id_sensor], c3.history['val_loss'][-1]

In [24]:
maes = {}
for i in range(len(longs_np)):
    print(i, longs_np[i])
    sensor, mae = train_and_test_sensor(i)
    maes[sensor] = mae 

0 AP7
MAE_val  75.80979736120578
MAE_val  51.144046569770296
MAE_val  23.682809347927705
MAE_val  48.48229516641444
MAE_val  77.36800727015067
MAE_test  56.7753012248093
1 AP6
MAE_val  74.61921432023048
MAE_val  49.396017061638695
MAE_val  22.354893112182616
MAE_val  45.92701640405692
MAE_val  76.46450659658674
MAE_test  56.88833060529433
2 AP4
MAE_val  66.64517080325065
MAE_val  44.6123261167234
MAE_val  22.224348633040396
MAE_val  46.57306654620193
MAE_val  70.25389557251026
MAE_test  53.94465374119383
3 AP3
MAE_val  53.7360220670414
MAE_val  40.76224770307509
MAE_val  21.362736480505344
MAE_val  73.05565114166914
MAE_val  46.59384850912657
MAE_test  6.656649084376051
4 AP5
MAE_val  55.24970045057928
MAE_val  36.355663239521924
MAE_val  22.087352481755055
MAE_val  37.961037127287135
MAE_val  57.43544904833747
MAE_test  41.17154105209058
5 AP1
MAE_val  53.48813401489421
MAE_val  41.610838796882916
MAE_val  20.758525616729365
MAE_val  41.51859642320799
MAE_val  60.26669131750933
MAE_te

In [25]:
maes = pd.Series(maes, name='MAE').sort_values()

In [26]:
maes

AP3      6.656649
DH8     40.139457
DH11    40.680962
AP5     41.171541
DH10    43.261680
DH6     43.378301
DH9     44.516583
DH7     45.890828
DH4     48.433724
DH5     49.663516
DH1     50.189021
DH3     50.249993
DH2     51.129463
AP1     51.893635
AP4     53.944654
AP7     56.775301
AP6     56.888331
Name: MAE, dtype: float64

# Convolutional 1D over time (longitude,time)

In this case, we are going to select more than one time slice as inputs. In this case X is a numpy array of rank 3.

X : (item,sensor,time_slice)

In [189]:
longs.index.to_numpy()

array(['AP7', 'AP6', 'AP4', 'AP3', 'AP5', 'AP1', 'DH5', 'DH3', 'DH4',
       'DH11', 'DH2', 'DH10', 'DH8', 'DH6', 'DH7', 'DH9', 'DH1'],
      dtype=object)

In [191]:
longs_np = longs.index.to_numpy()
n_sensors = len(longs_np)
print('Number of sensors considered:',n_sensors)

Number of sensors considered: 17


In [281]:
chosen_times = times[1:10:1]
print('Chosen times as features',chosen_times)

Chosen times as features ['t-1', 't-2', 't-3', 't-4', 't-5', 't-6', 't-7', 't-8', 't-9']


In [282]:
lr = 0.0001
lr = 0.0001
opt = keras.optimizers.Adam(lr=lr)

# We add a callback to log metrics and another one to schedule the learning rate

#see clr.py in this same folder
from clr import CyclicLR

c1 = keras.callbacks.BaseLogger(stateful_metrics=None)
c2 = CyclicLR(step_size=250, base_lr=lr)
c3 = keras.callbacks.History()

batch_size = 1 << 11   # as big as possible so we can explore many models
epochs = 1 << 5

In [283]:
def to_array(chosen_times, sensor='AP5', order_time = True):
    ''' Converts dataframe to numpy array for predicting any given sensor. '''
    
    X_train_np = []
    for time in chosen_times:
        X_train_np.append(X_train[time].to_numpy().reshape(-1,n_sensors,1))
    X_train_np = np.swapaxes(np.concatenate(X_train_np,axis=2),1,2)

    y_train_np = y_train[sensor].to_numpy()

    
    
    
    X_test_np = []
    for time in chosen_times:
        X_test_np.append(X_test[time].to_numpy().reshape(-1,n_sensors,1))
    X_test_np = np.swapaxes(np.concatenate(X_test_np,axis=2),1,2)
    y_test_np = y_test[sensor].to_numpy()
    
    if order_time:
        X_train_np = X_train_np[:,::-1]
        X_test_np = X_test_np[:,::-1]
    
    #return X_tr1_1_np[:val_idx], y_tr1_1_np[:val_idx], X_tr1_1_np[val_idx:], y_tr1_1_np[val_idx:], X_te1_1_np, y_te1_1_np
    return X_train_np, y_train_np, X_test_np, y_test_np

In [290]:
def make_model_sensor_conv_time(index_sensor, n_sensors=17, time_slices = 2):
    ''' Returns a model using all the sensors to predict index_sensor '''
    xin = Input(shape=(time_slices,n_sensors), name='main_input')
    '''
    When using this layer as the first layer in a model, provide an input_shape argument 
    (tuple of integers or None, does not include the batch axis), e.g. input_shape=(10, 128) 
    for time series sequences of 10 time steps with 128 features per step in data_format="channels_last", 
    or (None, 128) for variable-length sequences with 128 features per step.
    '''
    # The convolution is across time, and each sensor is a channel
    x = LocallyConnected1D(20, 3, data_format = 'channels_last', padding='valid')(xin)
    x = Activation('relu')(x)
    x = LocallyConnected1D(12, 3, data_format = 'channels_last', padding='valid')(x)
    x = Activation('relu')(x)
    x = Conv1D(32, 3, data_format = 'channels_last', padding='causal')(x)
    xl = Flatten()(x)
    xl = Dropout(0.2)(xl)
    xo = Dense(1)(xl)

    # use date info here?
    xinf = Flatten()(xin)
    s  = Dense(5)(xinf)
    s = Activation('tanh')(s)
    s = Dense(2)(s)
    s = Activation('softmax')(s)

    # sort of residual connection, we only take the last timestamp of the given sensor x[:,index_sensor,-1:]
    xin_0 = Activation('relu')(xin)
    xin_1 = Lambda(lambda x : x[:,index_sensor,-1:])(xin_0)
#     xin_1 = Flatten()(xin_1)
    xo_m = Dot(axes=1)([Concatenate()([xo,xin_1]), s])
    xo_m = Activation('relu')(xo_m)

    model = Model(inputs=[xin], outputs=[xo_m])
    return model

In [291]:
model = make_model_sensor_conv_time(0, n_sensors=n_sensors,time_slices=len(chosen_times))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 9, 17)        0                                            
__________________________________________________________________________________________________
locally_connected1d_248 (Locall (None, 7, 20)        7280        main_input[0][0]                 
__________________________________________________________________________________________________
activation_708 (Activation)     (None, 7, 20)        0           locally_connected1d_248[0][0]    
__________________________________________________________________________________________________
locally_connected1d_249 (Locall (None, 5, 12)        3660        activation_708[0][0]             
__________________________________________________________________________________________________
activation

In [292]:
def train_and_test_sensor(id_sensor=4):
    
    X_tr, y_tr, X_te, y_te = to_array(chosen_times,sensor=longs_np[id_sensor])
    print(X_tr.shape)
    # Validation using TS split (just to obtain different MAE estimations, no hyperoptimization for the moment)
    for tr_idx, va_idx in TimeSeriesSplit(n_splits=5).split(X_tr):
        model = make_model_sensor_conv_time(id_sensor, n_sensors=n_sensors,time_slices=len(chosen_times))
        model.compile(opt, loss='mean_absolute_error')
        
        model.fit(np.atleast_3d(X_tr[tr_idx]), y_tr[tr_idx], batch_size=batch_size, epochs=epochs, validation_data=
              (np.atleast_3d(X_tr[va_idx]),y_tr[va_idx]), callbacks=[c2, c3], verbose=0)
        print('MAE_val ', c3.history['val_loss'][-1])
    
    # Testing
    model = make_model_sensor_conv_time(id_sensor, n_sensors=n_sensors,time_slices=len(chosen_times))
    model.compile(opt, loss='mean_absolute_error')
    model.fit(np.atleast_3d(X_tr), y_tr, batch_size=batch_size, epochs=epochs, validation_data=
              (np.atleast_3d(X_te),y_te), callbacks=[c2, c3], verbose=0)
    
    print('MAE_test ', c3.history['val_loss'][-1])
    return longs_np[id_sensor], c3.history['val_loss'][-1]

0 AP7

MAE_val  75.80979736120578

MAE_val  51.144046569770296

MAE_val  23.682809347927705

MAE_val  48.48229516641444

MAE_val  77.36800727015067

MAE_test  56.7753012248093

In [293]:
i=0
print(i, longs_np[i])
sensor, mae = train_and_test_sensor(i)

0 AP7
(449885, 9, 17)


/home/davidgg/anaconda3/envs/solar/lib/python3.6/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.517210). Check your callbacks.
  % delta_t_median)


MAE_val  77.5054547008434
MAE_val  52.17530269164917
MAE_val  24.86432486753141
MAE_val  53.33971921361647
MAE_val  79.15222348050138
MAE_test  58.97714640068765


In [ ]:
maes = {}
for i in range(len(longs_np)):
    print(i, longs_np[i])
    sensor, mae = train_and_test_sensor(i)
    maes[sensor] = mae 

# Convolutional 2D

In [294]:
def to_array(chosen_times, sensor='AP5', order_time = True):
    ''' Converts dataframe to numpy array for predicting any given sensor. '''
    
    X_train_np = []
    for time in chosen_times:
        X_train_np.append(X_train[time].to_numpy().reshape(-1,n_sensors,1))
    X_train_np = np.swapaxes(np.concatenate(X_train_np,axis=2),1,2)

    y_train_np = y_train[sensor].to_numpy()

    
    
    
    X_test_np = []
    for time in chosen_times:
        X_test_np.append(X_test[time].to_numpy().reshape(-1,n_sensors,1))
    X_test_np = np.swapaxes(np.concatenate(X_test_np,axis=2),1,2)
    y_test_np = y_test[sensor].to_numpy()
    
    if order_time:
        X_train_np = np.expand_dims(X_train_np[:,::-1],axis=3)
        X_test_np = np.expand_dims(X_test_np[:,::-1],axis=3)
    
    #return X_tr1_1_np[:val_idx], y_tr1_1_np[:val_idx], X_tr1_1_np[val_idx:], y_tr1_1_np[val_idx:], X_te1_1_np, y_te1_1_np
    return X_train_np, y_train_np, X_test_np, y_test_np

In [302]:
def make_model_sensor_conv_time(index_sensor, n_sensors=17, time_slices = 2):
    ''' Returns a model using all the sensors to predict index_sensor '''
    xin = Input(shape=(time_slices,n_sensors,1), name='main_input')
    '''
    When using this layer as the first layer in a model, provide an input_shape argument 
    (tuple of integers or None, does not include the batch axis), e.g. input_shape=(10, 128) 
    for time series sequences of 10 time steps with 128 features per step in data_format="channels_last", 
    or (None, 128) for variable-length sequences with 128 features per step.
    '''
    # The convolution is across time, and each sensor is a channel
    x = LocallyConnected2D(20, 3, data_format = 'channels_last', padding='valid')(xin)
    x = Activation('relu')(x)
    x = LocallyConnected2D(12, 3, data_format = 'channels_last', padding='valid')(x)
    x = Activation('relu')(x)
    x = Conv2D(32, 3, data_format = 'channels_last', padding='valid')(x)
    xl = Flatten()(x)
    xl = Dropout(0.2)(xl)
    xo = Dense(1)(xl)

    # use date info here?
    xinf = Flatten()(xin)
    s  = Dense(5)(xinf)
    s = Activation('tanh')(s)
    s = Dense(2)(s)
    s = Activation('softmax')(s)

    # sort of residual connection, we only take the last timestamp of the given sensor x[:,index_sensor,-1:]
    xin_0 = Activation('relu')(xin)
    xin_1 = Lambda(lambda x : x[:,index_sensor,-1:,-1])(xin_0)
#     xin_1 = Flatten()(xin_1)
    xo_m = Dot(axes=1)([Concatenate()([xo,xin_1]), s])
    xo_m = Activation('relu')(xo_m)

    model = Model(inputs=[xin], outputs=[xo_m])
    return model

In [303]:
model = make_model_sensor_conv_time(0, n_sensors=n_sensors,time_slices=len(chosen_times))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 9, 17, 1)     0                                            
__________________________________________________________________________________________________
locally_connected2d_7 (LocallyC (None, 7, 15, 20)    21000       main_input[0][0]                 
__________________________________________________________________________________________________
activation_757 (Activation)     (None, 7, 15, 20)    0           locally_connected2d_7[0][0]      
__________________________________________________________________________________________________
locally_connected2d_8 (LocallyC (None, 5, 13, 12)    141180      activation_757[0][0]             
__________________________________________________________________________________________________
activation

In [304]:
def train_and_test_sensor(id_sensor=4):
    X_tr, y_tr, X_te, y_te = to_array(chosen_times,sensor=longs_np[id_sensor])
    print(X_tr.shape)
    # Validation using TS split (just to obtain different MAE estimations, no hyperoptimization for the moment)
    for tr_idx, va_idx in TimeSeriesSplit(n_splits=5).split(X_tr):
        model = make_model_sensor_conv_time(id_sensor, n_sensors=n_sensors,time_slices=len(chosen_times))
        model.compile(opt, loss='mean_absolute_error')
        
        model.fit(np.atleast_3d(X_tr[tr_idx]), y_tr[tr_idx], batch_size=batch_size, epochs=epochs, validation_data=
              (np.atleast_3d(X_tr[va_idx]),y_tr[va_idx]), callbacks=[c2, c3], verbose=0)
        print('MAE_val ', c3.history['val_loss'][-1])
    
    # Testing
    model = make_model_sensor_conv_time(id_sensor, n_sensors=n_sensors,time_slices=len(chosen_times))
    model.compile(opt, loss='mean_absolute_error')
    model.fit(np.atleast_3d(X_tr), y_tr, batch_size=batch_size, epochs=epochs, validation_data=
              (np.atleast_3d(X_te),y_te), callbacks=[c2, c3], verbose=0)
    
    print('MAE_test ', c3.history['val_loss'][-1])
    return longs_np[id_sensor], c3.history['val_loss'][-1]

0 AP7

MAE_val  75.80979736120578

MAE_val  51.144046569770296

MAE_val  23.682809347927705

MAE_val  48.48229516641444

MAE_val  77.36800727015067

MAE_test  56.7753012248093

In [305]:
i=0
print(i, longs_np[i])
sensor, mae = train_and_test_sensor(i)

0 AP7
(449885, 9, 17, 1)
MAE_val  76.37582733040334
MAE_val  58.20910274600245
MAE_val  29.512727100253265
MAE_val  48.84466819946338
MAE_val  77.00408122626646
MAE_test  56.87768337969694
